In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
driver.get('https://launchstudio.bluetooth.com/Listings/Search')

#Clicking to expand 'Advanced Search' form
adv_form = driver.find_element(By.LINK_TEXT, 'Advanced Search')
adv_form.click()

#Ticking additional type of product from 'Advanced Search' menu
project_type = driver.find_element(By.XPATH, '//*[@id="searchIn"]/label[3]/input')
project_type.click()

#Inserting dates into form
from_date = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div/div/div/div[2]/div/div/div[2]/div/div[5]/div/div[2]/input')
from_date.send_keys("2023-01-01")
to_date = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div/div/div/div[2]/div/div/div[2]/div/div[6]/div/div[2]/input')
to_date.send_keys("2023-09-13")

#Inserting keyword into search box
search_input = driver.find_element(By.XPATH,'//*[@id="listingsSearch"]/div/div[3]/div/div/div[1]/div/input')
search_input.send_keys("BT")

time.sleep(10)

#Clicking 'search' button
search_button = driver.find_element(By.ID,"searchButton")
search_button.click()

time.sleep(10)

#Finding number of pages with searched listings
table = []
pages_str = driver.find_element(By.XPATH, '/html/body/div[1]/div[3]/div/div/div/div[4]/div/div/div[1]/ul/li[6]/span/label[2]').text
pages_num = int(pages_str)
x = 1

#Iterating over each page
while x <= pages_num:
    wait = WebDriverWait(driver, 10)
    table_wait = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "table")))
    rows = driver.find_elements(By.XPATH, '//table[@class="table"]/tbody/tr')
    
    #Iterating over each row in the table
    for r in rows:
        data = r.find_elements(By.TAG_NAME, "td")
        
        #Extracting elements from the website
        if len(data) > 0:
            decl_id = data[0].find_element(By.TAG_NAME, "a").text
            qdids = data[1].text             
            comp = data[2].text
            prod = data[3].text
            spec = data[4].text
            date = data[5].text
            
            #Appending extracted elements to the list
            table.append({
                "declaration_id": decl_id,
                "QDID(s)": qdids,
                "Company": comp,
                "Products": prod,
                "Specification_code": spec,
                "Listing_date": date
                }) 
            
    #Clicking 'Next' button to go to the next page of listings
    next_button = driver.find_element(By.XPATH, '//*[@id="listingsSearch"]/div/div[4]/div/div/div[1]/ul/li[3]/button')
    next_button.click()
    x+=1
    
#Converting scraped data to Pandas data frame
df = pd.DataFrame(table)

#Saving data frame to csv file
df.to_csv(r'C:\Users\Nowy_użytkownik\Desktop\fbl-scraper-ok\launchstudio.csv',index=False, encoding='utf-8-sig', lineterminator='\n')